In [1]:
from ydc.tools.supercats import add_supercats
import import_data
import pandas as pd
from simplekml import Kml, Style
from colorsys import hsv_to_rgb

#Little helper function to restrict to one city because google earth is helpless otherwise
def get_by_city(city):
    return businesses[businesses['city']==city].copy(deep=True)

#second helper for color conv, copies from activestate.com and slightly modfied
def get_color(hue, saturation):
    rgb_tuple = hsv_to_rgb(hue, saturation,1)
    """ convert an (R, G, B) tuple to #RRGGBBff """
    rgb_tuple = (rgb_tuple[0]*255, rgb_tuple[1]*255, rgb_tuple[2]*255)
    hexcolor = '%02x%02x%02x' % rgb_tuple
    return hexcolor

In [2]:
# Import businesses
businesses = import_data.import_businesses()
(df, cats) = add_supercats(businesses)

In [6]:
def kml_by_city(city, df, cats):
    #Find supercats    
    # Prepare KML
    kml = Kml()

    # Create folder for businesses
    fol = kml.newfolder(name="Businesses")

    # Create subfolder for every category and save them in dictionary
    subfolders = {key: fol.newfolder(name=cats[key]['name']) for key in cats}

    # More subfolders for subcats
    subsubfolders = {}
    for superkey in subfolders:
        subcats = cats[superkey]['sub_categories']
        subsubfolders[superkey] = {key: subfolders[superkey].newfolder(name=subcats[key]['name']) for key in subcats}
        
    
    
    #Create color for every category and also save them in dict
    styles = {}
    for superkey in cats:
        subcats = cats[superkey]['sub_categories']
        # Create dict of dicts with entries for every subcategory
        styles[superkey] = {key: Style() for key in subcats}
        for key in subcats:
            styles[superkey][key].labelstyle.scale = 0
            hue = superkey/len(cats.keys())  # Around the circle
            sat = 0.5 + key/(2 * len(subcats.keys()))  # Avoid the middle so not everything gets white
            styles[superkey][key].iconstyle.icon.href = (
                'http://thydzik.com/thydzikGoogleMap/markerlink.php?color=%s' % get_color(hue, sat)
            )

    #Special case for uncategories (those are -1 and will be black)
    styles[-1] = {-1: Style()}
    styles[-1][-1].labelstyle.scale = 0
    styles[-1][-1].iconstyle.icon.href = 'http://thydzik.com/thydzikGoogleMap/markerlink.php?color=000000'
    
    for item in df.loc[df['city']==city ,('name', 'longitude', 'latitude','super_category','sub_category')].values:
        pnt = subsubfolders[item[3]][item[4]].newpoint(name=item[0], coords=[(item[1],item[2])])
        pnt.style = styles[item[3]][item[4]]

    kml.save("%s.kml" % city)

In [7]:
kml_by_city('Las Vegas', df,cats)

In [ ]:
type(df.loc[0,'sub_category'].astype(int))